# Project 3


# Movie Genre Classification

In [1]:
# Cargue librerias
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

Cargue de datasets

In [2]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Revisión de datos
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


### Create count vectorizer

Transformamos los textos con el uso de TfidfVectorizer

In [5]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

vect = TfidfVectorizer(strip_accents='unicode', token_pattern=r'\w{2,}',sublinear_tf=True, ngram_range=(4, 8),max_df=0.5,analyzer=split_into_lemmas,stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\leon-\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Create y

In [6]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

Partimos en train y test la base de datos

In [7]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

Cargue de librerias

In [8]:
! pip install livelossplot
from tensorflow import keras 
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from livelossplot import PlotLossesKeras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline

Entrenamiento del modelo - red neuronal

In [9]:
dims = X_train.shape[1]
output_var = y_train_genres.shape[1]
K.clear_session()
print("Building model...")
model = Sequential()
model.add(Dense(512, input_shape=(dims,),activation='exponential'))
model.add(Dense(output_var))
model.add(Activation('sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Building model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               17721344  
_________________________________________________________________
dense_1 (Dense)              (None, 24)                12312     
_________________________________________________________________
activation (Activation)      (None, 24)                0         
Total params: 17,733,656
Trainable params: 17,733,656
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_train, y_train_genres, validation_data=[X_test, y_test_genres], 
          batch_size=1000, epochs=25, verbose=1,
          )

Train on 5289 samples, validate on 2606 samples
Epoch 1/25
5289/5289 [==============================] - 10s 2ms/sample - loss: 0.4928 - binary_accuracy: 0.7722 - val_loss: 0.3041 - val_binary_accuracy: 0.8904
Epoch 2/25
5289/5289 [==============================] - 10s 2ms/sample - loss: 0.2863 - binary_accuracy: 0.8991 - val_loss: 0.2902 - val_binary_accuracy: 0.8850
Epoch 3/25
5289/5289 [==============================] - 10s 2ms/sample - loss: 0.2735 - binary_accuracy: 0.8964 - val_loss: 0.2854 - val_binary_accuracy: 0.8917
Epoch 4/25
5289/5289 [==============================] - 11s 2ms/sample - loss: 0.2648 - binary_accuracy: 0.8971 - val_loss: 0.2824 - val_binary_accuracy: 0.8935
Epoch 5/25
5289/5289 [==============================] - 11s 2ms/sample - loss: 0.2577 - binary_accuracy: 0.8977 - val_loss: 0.2794 - val_binary_accuracy: 0.8882
Epoch 6/25
5289/5289 [==============================] - 11s 2ms/sample - loss: 0.2499 - binary_accuracy: 0.9042 - val_loss: 0.2772 - val_binary_acc

### Predict the testing dataset

In [11]:
test = vect.transform(["On the Italian Riviera, an unlikely but strong friendship grows between a human being and a sea monster disguised as a human."])

y_pred = model.predict_proba(test)

d = {'p_Action': y_pred[0][1], 'p_Adventure': y_pred[0][1], 'p_Comedy': y_pred[0][4], 
         'p_Crime': y_pred[0][5], 'p_Documentary': y_pred[0][6], 'p_Drama': y_pred[0][7], 'p_Family': y_pred[0][8],
         'p_Horror': y_pred[0][12], 'p_Romance': y_pred[0][17], 'p_Sci-Fi':y_pred[0][18]}

print(d)

{'p_Action': 0.19274798, 'p_Adventure': 0.19274798, 'p_Comedy': 0.52252233, 'p_Crime': 0.07871005, 'p_Documentary': 0.015979648, 'p_Drama': 0.36437023, 'p_Family': 0.12927255, 'p_Horror': 0.08794251, 'p_Romance': 0.3356712, 'p_Sci-Fi': 0.14014903}
